# C3M1: Autograded Assignment

### Outline:
**Here are the objectives of this assignment:**

1. Familiarize yourself with odds and how they convert to probabilities.
2. Review Maximum Likelihood Estimates.
3. Understand the difference between Binomial Regression and Logistic Regression.
4. Get a basic understanding of Logistic regression models and properties.
5. Apply Logistic Regression techniques to real data.

**Here are some general tips:**

1. Read the questions carefully to understand what is being asked.
2. When you feel that your work is completed, feel free to hit the ```Validate``` button to see your results on the *visible* unit tests. If you have questions about unit testing, please refer to the "Module 0: Introduction" notebook provided as an optional resource for this course. In this assignment, there are hidden unit tests that check your code. You will not recieve any feedback for failed hidden unit tests until the assignment is submitted. **Do not misinterpret the feedback from visible unit tests as all possible tests for a given question--write your code carefully!**
3. Before submitting, we recommend restarting the kernel and running all the cells in order that they appear to make sure that there are no additional bugs in your code.

In [1]:
# Load necesary libraries
library(testthat)
library(tidyverse)
library(MASS)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::is_null() masks testthat::is_null()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::matches() masks tidyr::matches(), testthat::matches()


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select




# Problem 1: Logistic Regression Basics

Welcome to your first autograded assignment for Generalize Linear Models and Nonparametric Regression. Instead of throwing you directly into the code, let's start off slow with some conceptual questions. We will get to the actual coding part shortly.

### 1. (a) Odds and Ends

For each of the following questions, save your answer in the respecitve variable for that problem. You don't need to show your work, just submit your final answer.


1. What is the equivalent odds for the probability $0.25$?
2. You are testing a new drug and have gathered binary data on whether the drug performed its desired effects. From the control trial, 102 people saw improvement with a placebo and 241 did not. With the drug, 67 people saw improvement and 82 did not. What is the odds ratio of these results?
3. You've decided to determine the probability of a picture containing an animal on social media. On 6 different days, you look at 10 random pictures and record the number of pictures that contain at least one animal, and get the values $\{6, 10, 7, 9, 5, 9\}$. Given these results, what is the MLE for the probability of a picture containing an animal?

In [2]:
# Answer each with the correct numeric value.
prob.1.a.1 = 1/3

prob.1.a.2 = (67/82)/(102/241)

prob.1.a.3 = 1


In [3]:
# Test Cell 
# Be aware, there may be hidden tests that you don't see the answer to. 
# Even if your answers pass all the visible tests, there may be hidden tests that you're not passing.

### 1. (b) Logistic Regression TRUE/FALSE

For each of the following questions, save the boolean `TRUE` or `FALSE` (case sensitive) in the corresponding variable. 

1. Accuracy, Log-Loss and Mean-Squared Error are all evaulation metrics that can be used with Logistic Regression.
2. The Logit link function is defined as the log of the odds function. Therefor, the Logit function has a range of $[0, \infty]$.
3. Suppose you fit a Logistic Regression classifier to a response variable $\in \{0, 1\}$ and get $y = g(\beta_0 + \beta_1x_1 + \beta_2x_2)$ where $\beta_0 = 4, \beta_1=1, \beta_2=-2$ and $g()$ is the link function. Then the input $x_i=(1, 3)$ would be classified as $0$.

In [4]:
# Answer each with either TRUE or FALSE.
prob.1.b.1 = FALSE

prob.1.b.2 = FALSE

prob.1.b.3 = TRUE


In [5]:
# Test Cell
if(!test_that("Checking answer type", expect_is(prob.1.b.1, 'logical'))){
    print("Make sure your answers are booleans!")
}

# Problem 2: Bacteria growth in apple juice

Apparently, other organisms like apple juice too. So much so that some researchers decided to measure the growth of certain bacteria in different apple juice solutions. They measured whether different pH, temperature and molecular concentrations affected the growth of Alicyclobacillus Acidoterrestris CRA7152. 

Lets use their data to practice our Binomial (Logistic) modelling skills. We use the code cell below to load in the data.

In [6]:
library(ggplot2)

apple.data = read.csv(url(paste0("https://raw.githubusercontent.com/bzaharatos",
                                 "/-Statistical-Modeling-for-Data-Science-Applications/",
                                 "master/Generalized%20Linear%20Models%20and%20Nonparametric%20Regression/",
                                 "Datasets/apple_juice.dat")), sep = "")
    
names(apple.data) = c("pH", "nisin", "temp", "brix", "growth")
apple.data$growth = as.factor(apple.data$growth)

head(apple.data)

,pH,nisin,temp,brix,growth
,<dbl>,<int>,<int>,<int>,<fct>
1,5.5,70,43,19,0
2,5.5,50,43,13,1
3,5.5,50,35,15,1
4,5.5,30,35,13,1
5,5.5,30,25,11,0
6,5.5,0,50,19,0


In [7]:
str(apple.data)

'data.frame':	73 obs. of  5 variables:
 $ pH    : num  5.5 5.5 5.5 5.5 5.5 5.5 5.5 3.5 3.5 3.5 ...
 $ nisin : int  70 50 50 30 30 0 0 70 70 50 ...
 $ temp  : int  43 43 35 35 25 50 25 43 35 50 ...
 $ brix  : int  19 13 15 13 11 19 15 15 11 13 ...
 $ growth: Factor w/ 2 levels "0","1": 1 2 2 2 1 1 2 1 1 1 ...


### 2. (a) Creating the Model

Fit a logistic regression model to the data, with `growth` as the response and all other variables as the predictors. Save this model as `glmod.apple`. Further, save $\widehat\beta_{temp}$, the parameter estimate for `temp`, in `beta_temp`.

In [8]:
# Fit the logistic regression model
glmod.apple <- glm(growth ~ pH + nisin + temp + brix, data = apple.data, family = binomial)

# Extract the parameter estimate for `temp`
beta_temp <- coef(glmod.apple)["temp"]

# Display the model summary
summary(glmod.apple)

# Display the value of beta_temp
beta_temp



Call:
glm(formula = growth ~ pH + nisin + temp + brix, family = binomial, 
    data = apple.data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3245  -0.4325  -0.1415   0.5308   1.5593  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -7.68363    3.28201  -2.341 0.019225 *  
pH           2.04908    0.57481   3.565 0.000364 ***
nisin       -0.06273    0.01910  -3.283 0.001026 ** 
temp         0.12532    0.05079   2.467 0.013614 *  
brix        -0.38000    0.15909  -2.389 0.016915 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 95.072  on 72  degrees of freedom
Residual deviance: 49.844  on 68  degrees of freedom
AIC: 59.844

Number of Fisher Scoring iterations: 6


temp 
0.1253159

In [9]:
# Test Cell

### 2. (b) The Effects of Temperature

How might we understand the relationship between a specific predictor in the model, say, temperature, and the odds of growth? 

- Our model gives us one answer to this question. Recall that $\widehat\beta_{temp}$ can be interpreted as the (additive) change in the log odds of success for a one-unit increase in temperature, holding all other predictors constant. 

- Similarly, $e^{\widehat\beta_{temp}}$ can be interpreted as the *multiplicitive* change in the odds of success for a one-unit increase in temperature, holding all other predictors constant. 

Let's verify the interpretation of $e^{\widehat\beta_{temp}}$ using the following steps:

1. Calculate the mean of each predictor column. Store those means in a ($1 \times 4$) data frame named `df_means`. Make sure that the columns of `df_means` are named the same as the predictors in the data frame. 


2. Using `df_means`, compute the odds of growth at the mean value for each predictor. Store this value in `e1`. HINT: `predict(glmod.apple, new = df_means)` will give this answer on the log-odds scale.


3. Compute `beta_temp*e1`, using `beta_temp` from part (a).


4. Now, construct a new data frame, `df_means1`, that is the same as `df_means` except that the `temp` value is one-unit larger than its mean (all other columns should remain at their mean data value).


5. Compute the odds of growth at the predictor values given in `df_means1`. Store this computation in `e2`.

Note that `e2` should equal `beta_temp*e1`!

In [10]:
# Step 1: Calculate the mean of each predictor column and store them in a data frame named df_means
df_means <- data.frame(
  pH = mean(apple.data$pH),
  nisin = mean(apple.data$nisin),
  temp = mean(apple.data$temp),
  brix = mean(apple.data$brix)
)

# Step 2: Compute the odds of growth at the mean value for each predictor, stored in e1
log_odds_e1 <- predict(glmod.apple, newdata = df_means)
e1 <- exp(log_odds_e1)

# Step 3: Compute beta_temp * e1
beta_temp_e1 <- beta_temp * e1

# Step 4: Construct a new data frame df_means1 where temp is one unit larger than its mean
df_means1 <- df_means
df_means1$temp <- df_means$temp + 1

# Step 5: Compute the odds of growth at the predictor values given in df_means1, stored in e2
log_odds_e2 <- predict(glmod.apple, newdata = df_means1)
e2 <- exp(log_odds_e2)

# Display the results
df_means
df_means1
e1
beta_temp_e1
e2



pH,nisin,temp,brix
<dbl>,<dbl>,<dbl>,<dbl>
4.472603,34.65753,38.21918,14.28767


pH,nisin,temp,brix
<dbl>,<dbl>,<dbl>,<dbl>
4.472603,34.65753,39.21918,14.28767


1 
0.2637765

temp 
0.03305538

1 
0.2989924

In [11]:
# Test Cell

### 2. (c) Quantifying uncertainty in parameters

Remember, in our logistic regression model, we assume that our response data come from a binomial distribution, which means that there is some inherent randomness in our measurements. Thus, a point estimate of a parameter is only so helpful. We might also want to quantify our uncertainty in a given parameter.

Calculate the $95\%$ confidence interval for $\beta_{temp}$. Use the assumption that $\widehat\beta_{temp} \overset{\text{approx}}{\sim} N(\beta_{temp}, se^2)$, where $se = 0.05079$ is the standard error in the summary table from part (a). Store the lower bound in `beta_temp_lower` and the upper bound in `beta_temp_upper`.

In [12]:
# Given values
se <- 0.05079

# Calculate the lower and upper bounds of the 95% confidence interval
beta_temp_lower <- beta_temp - 1.96 * se
beta_temp_upper <- beta_temp + 1.96 * se

# Display the results
beta_temp_lower
beta_temp_upper


temp 
0.02576746

temp 
0.2248643

In [13]:
# Test Cell